# LATEST - BBox

In [17]:

class LR1Parser:
    def __init__(self):
        self.terminals = set()
        self.non_terminals = set()
        self.productions = []
        self.start_symbol = None
        self.parsing_table = {}
        self.canonical_collection = []
        
    def _compute_first_sets(self) -> Dict[Symbol, Set[Symbol]]:
        first_sets = {sym: set() for sym in self.terminals.union(self.non_terminals)}
        
        # Terminals have themselves as first set
        for t in self.terminals:
            first_sets[t] = {t}
        
        # Compute first sets
        changed = True
        while changed:
            changed = False
            for prod in self.productions:
                # Add first of first symbol in production's right side
                if not prod.right:  # Epsilon production
                    if Symbol('ε', True) not in first_sets[prod.left]:
                        first_sets[prod.left].add(Symbol('ε', True))
                        changed = True
                elif prod.right[0].is_terminal:
                    if prod.right[0] not in first_sets[prod.left]:
                        first_sets[prod.left].add(prod.right[0])
                        changed = True
                else:
                    # For non-terminals, add their first sets
                    first_of_first = first_sets[prod.right[0]]
                    for first_sym in first_of_first:
                        if first_sym not in first_sets[prod.left]:
                            first_sets[prod.left].add(first_sym)
                            changed = True
        
        return first_sets

    def _compute_follow_sets(self, first_sets: Dict[Symbol, Set[Symbol]]) -> Dict[Symbol, Set[Symbol]]:
        follow_sets = {nt: set() for nt in self.non_terminals}
        
        # Start symbol has end marker
        follow_sets[self.start_symbol].add(Symbol('$', True))
        
        changed = True
        while changed:
            changed = False
            for prod in self.productions:
                for i, sym in enumerate(prod.right):
                    if not sym.is_terminal:
                        # Find what can follow this non-terminal
                        if i == len(prod.right) - 1:
                            # If at end of production, add follow of left side
                            for follow_sym in follow_sets[prod.left]:
                                if follow_sym not in follow_sets[sym]:
                                    follow_sets[sym].add(follow_sym)
                                    changed = True
                        else:
                            # Add first set of next symbol
                            next_sym = prod.right[i+1]
                            if next_sym.is_terminal:
                                if next_sym not in follow_sets[sym]:
                                    follow_sets[sym].add(next_sym)
                                    changed = True
                            else:
                                # Add first set of next non-terminal
                                for first_sym in first_sets[next_sym]:
                                    if first_sym.name != 'ε' and first_sym not in follow_sets[sym]:
                                        follow_sets[sym].add(first_sym)
                                        changed = True
        
        return follow_sets

    def _create_json_grammar(self):
        # Define terminals
        self.terminals = {
            Symbol('{', True), Symbol('}', True), Symbol('[', True), Symbol(']', True),
            Symbol(':', True), Symbol(',', True), 
            Symbol('string', True), Symbol('number', True), 
            Symbol('true', True), Symbol('false', True), Symbol('null', True)
        }

        # Define non-terminals
        self.non_terminals = {
            Symbol('JSON', False), Symbol('Object', False), Symbol('Array', False),
            Symbol('Pair', False), Symbol('Value', False), Symbol('Elements', False)
        }

        # Set start symbol
        self.start_symbol = Symbol('JSON', False)

        # Define productions
        self.productions = [
            Production(Symbol('JSON', False), [Symbol('Object', False)]),
            Production(Symbol('JSON', False), [Symbol('Array', False)]),
            
            Production(Symbol('Object', False), [Symbol('{', True), Symbol('Pair', False), Symbol('}', True)]),
            Production(Symbol('Object', False), [Symbol('{', True), Symbol('Pair', False), Symbol(',', True), Symbol('Pair', False), Symbol('}', True)]),
            
            Production(Symbol('Pair', False), [Symbol('string', True), Symbol(':', True), Symbol('Value', False)]),
            Production(Symbol('Pair', False), [Symbol('Pair', False), Symbol(',', True), Symbol('string', True), Symbol(':', True), Symbol('Value', False)]),
            
            Production(Symbol('Array', False), [Symbol('[', True), Symbol('Value', False), Symbol(']', True)]),            
            Production(Symbol('Array', False), [Symbol('[', True), Symbol('Value', False), Symbol(',', True), Symbol('Value', False), Symbol(']', True)]),
            
            Production(Symbol('Elements', False), [Symbol('Value', False)]),
            Production(Symbol('Elements', False), [Symbol('Elements', False), Symbol(',', True), Symbol('Value', False)]),
            
            Production(Symbol('Value', False), [Symbol('string', True)]),
            Production(Symbol('Value', False), [Symbol('number', True)]),
            Production(Symbol('Value', False), [Symbol('Object', False)]),
            Production(Symbol('Value', False), [Symbol('Array', False)]),
            Production(Symbol('Value', False), [Symbol('true', True)]),
            Production(Symbol('Value', False), [Symbol('false', True)]),
            Production(Symbol('Value', False), [Symbol('null', True)])
        ]

    def __init__(self):
        self.terminals = set()
        self.non_terminals = set()
        self.productions = []
        self.start_symbol = None
        self.parsing_table = {}
        self.canonical_collection = []
        
    def _closure(self, items: Set[LRItem], first_sets: Dict[Symbol, Set[Symbol]]) -> Set[LRItem]:
        closure = set(items)

        changed = True
        while changed:
            changed = False
            for item in list(closure):
                # If dot is before a non-terminal
                if item.dot_position < len(item.production.right) and not item.production.right[item.dot_position].is_terminal:
                    sym = item.production.right[item.dot_position]
                    beta_first = set()

                    # Compute FIRST(β lookahead) if there's more in the production
                    if item.dot_position + 1 < len(item.production.right):
                        beta = item.production.right[item.dot_position + 1:]
                        for b_sym in beta:
                            beta_first.update(first_sets.get(b_sym, set()))
                            if Symbol('ε', True) not in first_sets[b_sym]:
                                break
                    else:
                        beta_first.add(item.lookahead)

                    # Add new LR items
                    lookaheads = beta_first or {item.lookahead}
                    for prod in [p for p in self.productions if p.left == sym]:
                        for lookahead in lookaheads:
                            new_item = LRItem(prod, 0, lookahead)
                            if new_item not in closure:
                                closure.add(new_item)
                                changed = True
        return closure


    def _goto(self, items: Set[LRItem], symbol: Symbol, first_sets: Dict[Symbol, Set[Symbol]]) -> Set[LRItem]:
        goto_items = set()
        
        for item in items:
            # If dot is before the given symbol
            if (item.dot_position < len(item.production.right) and 
                item.production.right[item.dot_position] == symbol):
                # Create new item with dot moved
                new_item = LRItem(item.production, item.dot_position + 1, item.lookahead)
                goto_items.add(new_item)
        
        return self._closure(goto_items, first_sets)

    def _build_canonical_collection(self):
        first_sets = self._compute_first_sets()
        
        # Start with initial item
        initial_production = [p for p in self.productions if p.left == self.start_symbol][0]
        initial_item = LRItem(initial_production, 0, Symbol('$', True))
        initial_state = self._closure({initial_item}, first_sets)

        
        self.canonical_collection = [initial_state]
        work_list = [initial_state]
        
        while work_list:
            current_state = work_list.pop(0)
            
            # Find possible transitions
            symbols = set()
            for item in current_state:
                if item.dot_position < len(item.production.right):
                    symbols.add(item.production.right[item.dot_position])
            
            for symbol in symbols:
                goto_state = self._goto(current_state, symbol, first_sets)
                
                # Check if this state already exists
                if goto_state and goto_state not in self.canonical_collection:
                    self.canonical_collection.append(goto_state)
                    work_list.append(goto_state)

    def _build_parsing_table(self):
        self._build_canonical_collection()
        first_sets = self._compute_first_sets()
        follow_sets = self._compute_follow_sets(first_sets)

        # Initialize parsing table
        self.parsing_table = {}
        for state_index, state in enumerate(self.canonical_collection):
            self.parsing_table[state_index] = {}
            for item in state:
                if item.dot_position == len(item.production.right):  # Reduction
                    if item.production.left == self.start_symbol:  # Accept action
                        self.parsing_table[state_index][Symbol('$', True)] = ('accept', None)
                    else:
                        # Use Follow Set for reductions
                        for follow in follow_sets[item.production.left]:
                            if follow not in self.parsing_table[state_index]:
                                self.parsing_table[state_index][follow] = ('reduce', item.production)
                elif item.dot_position < len(item.production.right):  # Shift
                    symbol = item.production.right[item.dot_position]
                    goto_state = self._goto(state, symbol, first_sets)
                    if goto_state in self.canonical_collection:
                        goto_index = self.canonical_collection.index(goto_state)
                        self.parsing_table[state_index][symbol] = ('shift', goto_index)


    def parse(self, tokens):
        # Ensure parsing table is built
        if not self.parsing_table:
            self._build_parsing_table()
        
        # Convert tokens to symbols
        token_symbols = [Symbol(token[1], True) for token in tokens]
        token_symbols.append(Symbol('$', True))
        
        # Initialize parse stack
        stack = [0]  # Start with initial state
        input_ptr = 0
        
        while True:
            current_state = stack[-1]
            current_token = token_symbols[input_ptr]
            
            print(f"Current State: {current_state}, Current Token: {current_token}")
            print(f"Parsing Action: {self.parsing_table.get(current_state, {}).get(current_token, 'Error')}")
            
            # Look up action in parsing table
            if current_token not in self.parsing_table[current_state]:
                # Error handling
                raise ValueError(f"Syntax error: Unexpected token {current_token}")
            
            action, details = self.parsing_table[current_state][current_token]
            
            if action == 'shift':
                stack.append(current_token)
                stack.append(details)  # Push state
                input_ptr += 1
            elif action == 'reduce':
                # Pop states and symbols
                pop_count = len(details.right) * 2
                stack = stack[:-pop_count]
                
                # Push non-terminal and go to goto state
                stack.append(details.left)
                goto_state = self.parsing_table[stack[-2]][details.left][1]
                stack.append(goto_state)
            elif action == 'accept':
                return True  # Parsing successful
            
    def visualize_parse_tree(self, tokens):
        """
        Visualize the parse tree for a given input with more accurate parsing.

        Args:
            tokens (List[Tuple]): List of parsed tokens

        Returns:
            graphviz.Digraph: A visual representation of the parse tree
        """
        dot = graphviz.Digraph(comment='Parse Tree', format='png')
        dot.attr(rankdir='TB')  # Top to Bottom layout

        # Track parse tree construction during parsing
        

        def parse_with_tree_construction():
            stack = [0]  # Start with initial state
            input_ptr = 0
            token_symbols = [Symbol(token[1], True) for token in tokens]
            token_symbols.append(Symbol('$', True))

            # Track current production hierarchy
            production_stack = []
            parse_tree_nodes = []
            while True:
                current_state = stack[-1]
                current_token = token_symbols[input_ptr]

                if current_token not in self.parsing_table[current_state]:
                    raise ValueError(f"Syntax error: Unexpected token {current_token}")

                action, details = self.parsing_table[current_state][current_token]

                if action == 'shift':
                    # Create node for terminal
                    node_id = f"token_{input_ptr}"
                    dot.node(node_id, current_token.name, shape='ellipse', style='filled', fillcolor='lightgreen')

                    # Connect to parent if exists
                    if production_stack:
                        dot.edge(str(id(production_stack[-1])), node_id)

                    # Add to tracking
                    parse_tree_nodes.append((node_id, current_token))

                    stack.append(current_token)
                    stack.append(details)
                    input_ptr += 1

                elif action == 'reduce':
                    # Create node for production
                    node_id = str(id(details))
                    dot.node(node_id, details.left.name, shape='box', style='filled', fillcolor='lightyellow')

                    # Connect children
                    child_count = len(details.right)
                    children = parse_tree_nodes[-child_count:]
                    for child_node_id, _ in children:
                        dot.edge(node_id, child_node_id)

                    # Remove used children
                    parse_tree_nodes = parse_tree_nodes[:-child_count]
                    parse_tree_nodes.append((node_id, details.left))

                    # Manage production stack for hierarchy
                    production_stack.append(details)

                    # Pop states and symbols
                    pop_count = len(details.right) * 2
                    stack = stack[:-pop_count]

                    # Push non-terminal and go to goto state
                    stack.append(details.left)
                    goto_state = self.parsing_table[stack[-2]][details.left][1]
                    stack.append(goto_state)

                elif action == 'accept':
                    break
                
        # Run parsing with tree construction
        parse_with_tree_construction()

        # Render the graph
        dot.render('parse_tree', view=True)
        return dot
    
    def get_intermediate_representation(self, tokens):
        """
        Generate an intermediate representation of the parsing process.

        Args:
            tokens (List[Tuple]): List of parsed tokens

        Returns:
            Dict: A dictionary representing the parsing stages and intermediate structures
        """
        intermediate_repr = {
            'tokens': tokens,
            'parsing_stages': [],
            'final_ast': None
        }

        def create_ast_node(production, children=None):
            """Create an abstract syntax tree node"""
            return {
                'type': production.left.name,
                'children': children or []
            }

        def parse_with_stages():
            stack = [0]  # Start with initial state
            input_ptr = 0
            token_symbols = [Symbol(token[1], True) for token in tokens]
            token_symbols.append(Symbol('$', True))

            current_ast = None

            while True:
                current_state = stack[-1]
                current_token = token_symbols[input_ptr]

                # Capture current parsing stage
                intermediate_repr['parsing_stages'].append({
                    'state': current_state,
                    'token': current_token,
                    'stack': stack.copy(),
                    'input_ptr': input_ptr
                })

                if current_token not in self.parsing_table[current_state]:
                    raise ValueError(f"Syntax error: Unexpected token {current_token}")

                action, details = self.parsing_table[current_state][current_token]

                if action == 'shift':
                    stack.append(current_token)
                    stack.append(details)
                    input_ptr += 1
                elif action == 'reduce':
                    # Create AST node for reduction
                    child_nodes = []
                    pop_count = len(details.right) * 2
                    stack = stack[:-pop_count]

                    stack.append(details.left)
                    current_ast = create_ast_node(details)
                    goto_state = self.parsing_table[stack[-2]][details.left][1]
                    stack.append(goto_state)
                elif action == 'accept':
                    intermediate_repr['final_ast'] = current_ast
                    break
                
        parse_with_stages()
        return intermediate_repr
    

def tokenize_json(json_string):
    # Implement JSON tokenization
    token_patterns = [
        (r'\s+', None),  # Whitespace
        (r'\{', '{'),
        (r'\}', '}'),
        (r'\[', '['),
        (r'\]', ']'),
        (r':', ':'),
        (r',', ','),
        (r'"(?:\\.|[^"\\])*"', 'string'),  # Handle escaped quotes
        (r'-?\d+(\.\d+)?([eE][+-]?\d+)?', 'number'),
        (r'true', 'true'),
        (r'false', 'false'),
        (r'null', 'null')
    ]
    tokens = []
    while json_string:
        match = None
        for pattern, token_type in token_patterns:
            regex = re.compile(pattern)
            match = regex.match(json_string)
            if match:
                text = match.group(0)
                if token_type:
                    tokens.append((text, token_type))
                json_string = json_string[len(text):].lstrip()
                break
        
        if not match:
            raise ValueError(f"Unexpected token in JSON: {json_string}")
    
    return tokens


In [ ]:

def main():
    parser = LR1Parser()
    parser._create_json_grammar()
    with open('test.json', 'r', encoding='utf-8') as f:
        json_str = f.read()

    tokens = tokenize_json(json_str)
    print("Tokens:", tokens)

    try:
        result = parser.parse(tokens)
        print("Result:", result,"\n")
        print("Parsing successful!\n")
        parser.visualize_parse_tree(tokens)
        ir = parser.get_intermediate_representation(tokens) 
        print("Intermediate Representation:\n\n", ir)
        
    except ValueError as e:
        print(f"Parsing error: {e}")

if __name__ == "__main__":
    main()


Tokens: [('{', '{'), ('"name"', 'string'), (':', ':'), ('"John Doe"', 'string'), (',', ','), ('"age"', 'string'), (':', ':'), ('30', 'number'), (',', ','), ('"cities"', 'string'), (':', ':'), ('[', '['), ('"New York"', 'string'), (',', ','), ('"London"', 'string'), (']', ']'), (',', ','), ('"address"', 'string'), (':', ':'), ('{', '{'), ('"street"', 'string'), (':', ':'), ('"123 Main St"', 'string'), (',', ','), ('"city"', 'string'), (':', ':'), ('"Boston"', 'string'), (',', ','), ('"country"', 'string'), (':', ':'), ('"USA"', 'string'), ('}', '}'), ('}', '}')]
Current State: 0, Current Token: Terminal({)
Parsing Action: ('shift', 1)
Current State: 1, Current Token: Terminal(string)
Parsing Action: ('shift', 3)
Current State: 3, Current Token: Terminal(:)
Parsing Action: ('shift', 5)
Current State: 5, Current Token: Terminal(string)
Parsing Action: ('shift', 10)
Current State: 10, Current Token: Terminal(,)
Parsing Action: ('reduce', Value → string)
Current State: 12, Current Token: Te